### **Library Imports**

In [1]:
import os
import re
import torch
import numpy as np
import pandas as pd

from time import time
from torch import nn, optim
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
from torchvision import transforms, datasets

### **Configuration**

In [2]:
class CFG(object):
    def __init__(self, 
                 seed: int = 42,
                 size: int = 224,
                 n_splits: int = 5,
                 batch_size: int = 16,
                 epochs: int = 25,
                 early_stopping: int = 5,
                 lr: float = 1e-4,
                 wd: float = 0.0,
                 max_lr: float = 1e-3,
                 pct_start: float = 0.2,
                 steps_per_epoch: int = 100,
                 div_factor: int = 1e3, 
                 final_div_factor: float = 1e3,
                 ):
        self.seed = seed
        self.size = size
        self.n_splits = n_splits
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.lr = lr
        self.wd = wd
        self.max_lr = max_lr
        self.pct_start = pct_start
        self.steps_per_epoch = steps_per_epoch
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # self.train_transform = transforms.Compose([
        #     transforms.ToTensor(),
        #     transforms.Normalize([0.1307,], [0.3081,]),
        #     transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
        #     transforms.RandomHorizontalFlip(p=0.25),
        #     transforms.RandomVerticalFlip(p=0.25),
        # ])
        self.train_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.1307,], [0.3081,]),
        ])
        self.valid_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.1307,], [0.3081,]),
        ])
                                
        self.save_path = "saves"
        if not os.path.exists(self.save_path): os.makedirs(self.save_path)
    
cfg = CFG(seed=42)

### **Model**

In [3]:
class Model(nn.Module):
    def __init__(self, filter_sizes: list=[64, 128, 256], HL: list=[1024], DP: float=0.25):
        
        super(Model, self).__init__()
        
        self.features = nn.Sequential()
        self.features.add_module("CN1", nn.Conv2d(in_channels=1, out_channels=filter_sizes[0], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
        self.features.add_module("BN1", nn.BatchNorm2d(num_features=filter_sizes[0], eps=1e-5))
        self.features.add_module("AN1", nn.ReLU())
        self.features.add_module("MP1", nn.MaxPool2d(kernel_size=(2, 2)))
        self.features.add_module("CN2", nn.Conv2d(in_channels=filter_sizes[0], out_channels=filter_sizes[1], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
        self.features.add_module("BN2", nn.BatchNorm2d(num_features=filter_sizes[1], eps=1e-5))
        self.features.add_module("AN2", nn.ReLU())
        self.features.add_module("MP2", nn.MaxPool2d(kernel_size=(2, 2)))
        self.features.add_module("CN3", nn.Conv2d(in_channels=filter_sizes[1], out_channels=filter_sizes[2], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
        self.features.add_module("BN3", nn.BatchNorm2d(num_features=filter_sizes[2], eps=1e-5))
        self.features.add_module("AN3", nn.ReLU())
        self.features.add_module("MP3", nn.MaxPool2d(kernel_size=(2, 2)))    

        self.classifier = nn.Sequential()
        if len(HL) == 0:
            self.classifier.add_module("FC1", nn.Linear(in_features=filter_sizes[2]*3*3, out_features=10))
        elif len(HL) == 1:
            self.classifier.add_module("FC1", nn.Linear(in_features=filter_sizes[2]*3*3, out_features=HL[0]))
            if isinstance(DP, float):
                self.classifier.add_module("DP1", nn.Dropout(p=DP))
            self.classifier.add_module("AN1", nn.ReLU())
            self.classifier.add_module("FC2", nn.Linear(in_features=HL[0], out_features=10))
        elif len(HL) == 2:
            self.classifier.add_module("FC1", nn.Linear(in_features=filter_sizes[2]*3*3, out_features=HL[0]))
            if isinstance(DP, float):
                self.classifier.add_module("DP1", nn.Dropout(p=DP))
            self.classifier.add_module("AN1", nn.ReLU())
            self.classifier.add_module("FC2", nn.Linear(in_features=HL[0], out_features=HL[1]))
            if isinstance(DP, float):
                self.classifier.add_module("DP2", nn.Dropout(p=DP))
            self.classifier.add_module("AN2", nn.ReLU())
            self.classifier.add_module("FC3", nn.Linear(in_features=HL[1], out_features=10))
        
        self.classifier.add_module("Final Activation", nn.LogSoftmax(dim=1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        return self.classifier(x)

### **Fit and Predict Helpers**

In [4]:
def predict_batch(
    model_1=None,
    model_2=None,
    path_1=None, 
    path_2=None,
    weight_1=1.0, 
    weight_2=1.0,
    dataloader=None, 
    device=None
) -> np.ndarray:
    
    model_1.load_state_dict(torch.load(path_1, map_location=device)["model_state_dict"])
    model_1.to(device)    
    model_1.eval()

    model_2.load_state_dict(torch.load(path_2, map_location=device)["model_state_dict"])
    model_2.to(device)    
    model_2.eval()
    
    y_pred = torch.zeros(1, 1).to(device)
    
    for X, _ in dataloader:
        X = X.to(device)
        with torch.no_grad():
            output = torch.argmax(weight_1 * torch.exp(model_1(X)) + weight_2 * torch.exp(model_2(X)), dim=1)
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

### **Submission**

In [5]:
ts_data_setup = datasets.MNIST(
    root = "data", 
    train = False, 
    download = True,
    transform = cfg.valid_transform,
)
ts_data = DL(ts_data_setup, batch_size=cfg.batch_size, shuffle=False)

model = Model(filter_sizes=[64, 128, 256], HL=[4096]).to(cfg.device)

y_pred = predict_batch(
    model_1=model,
    model_2=model,
    path_2="../input/mbaza-cnn-5-s3407/saves/bae_state.pt",
    path_1="../input/mbaza-cnn-5/saves/ble_state.pt",
    weight_1=1.0,
    weight_2=1.0,
    dataloader=ts_data,
    device=cfg.device
)


ss_df = pd.DataFrame({
    "Id" : pd.RangeIndex(1,10001),
    "Category" : y_pred.reshape(-1).astype("uint8")
})
ss_df.to_csv("submission.csv", index=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

